In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy.integrate import odeint
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm

## Model

<img src="data/BA-Project-Model copy.png"/>

### New York Data

In [152]:
#### source https://github.com/midas-network/COVID-19/tree/master/parameter_estimates/2019_novel_coronavirus
N = 19.4e6
g_0 = 1/6
g_1 = 1/6
g_2 = 1/6
g_3 = 1/8
f = 0.3
a = 1/5.2
p1 = 1/6
p2 = 1/6
mu = 1/8
cfr = 0.012
d_1 = 0.8
d_2 = 0.15/0.2
d_3 = 0.05/(0.05+cfr)

In [153]:
new_york = pd.read_csv('data/master_data/new_york/from_wiki.csv', index_col=0)


fig = go.Figure()

fig.add_trace(go.Scatter(x=new_york.index, y=np.log(new_york['cases'].values),\
                         mode='markers' ))

fig.update_layout(title='Log(Infected) over time',
                   xaxis_title='Time',
                   yaxis_title='log(I)',
                 title_x=0.5,
                  width=800,
                  height=600)

fig.show()

In [154]:
subset = new_york.loc['2020-03-08':'2020-03-24']

X = np.arange(1, len(subset) + 1)
y = np.log(subset['cases'])

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

In [155]:
R_0 = round((model.params[1] + g_0)/g_0, 2)

In [156]:
b_1 = b_0 = round(R_0 * g_0, 2)

In [157]:
b = np.array([b_0, b_1, 0, 0])  ### assuming patients admitted in hospitals do not spread the disease
g = np.array([g_0, g_1, g_2, g_3])
p = np.array([p1, p2])
d = np.array([d_1, d_2, d_3])

In [158]:
def model(y, t, N, b, a, f, p, g, d, mu ):
    S, E, I0, I1, I2, I3, R, D = y 
    dy = [0]*8
    
    dy[0] = -np.dot(b, np.array([I0, I1, I2, I3]))*S/N
    dy[1] = np.dot(b, np.array([I0, I1, I2, I3]))*S/N - a*E
    dy[2] = a*f*E - g[0]*I0
    dy[3] = a*(1 - f)*E - g[1]*d[0]*I1 - p[0]*(1 - d[0])*I1
    dy[4] = p[0]*(1 - d[0])*I1 - g[2]*d[1]*I2 - p[1]*(1 - d[1])*I2
    dy[5] = p[1]*(1 - d[2])*I2 - g[3]*d[2]*I3 - mu*(1 - d[2])*I3
    dy[6] = np.dot(g, np.array([I0, d[0]*I1, d[1]*I2, d[2]*I3]))
    dy[7] = mu*(1 - d[2])*I3
    
    return dy

In [159]:
### Initial parameters
S, E, I0, I1, I2, I3, R, D = N-1, 1, 0, 0, 0, 0, 0, 0
y0 = S, E, I0, I1, I2, I3, R, D
max_t = 365
vect = np.linspace(0, max_t, max_t)
Ns = np.ones_like(vect) * N

ret = odeint(model, y0, vect, args=(N, b, a, f, p, g, d, mu))

S, E, I0, I1, I2, I3, R, D = ret.T

### Plot the output

In [160]:
fig = go.Figure()
labels = ['Susceptible', 'Exposed', 'Asymptomatic infections', 'mild infections', 'severe infections', \
'critical infections', 'recovered', 'dead', 'Total infections', 'Require Hospitalization']
colors = ['orange', 'red', 'red', 'red', 'darkred', 'darkred', 'green', 'darkred', 'red', 'darkred']
alphas = [0.8, 0.2, 0.35, 0.7, 0.7, 0.8, 0.8, 0.8, 0.9, 1.0]
widths = [2]* len(labels)
widths[-1] = 4
total_infected = I0 + I1 + I2 + I3
require_hospitalization = I2 + I3
count = 0
for elem, name in zip([S, E, I0, I1, I2, I3, R, D, total_infected, require_hospitalization], labels):
    fig.add_trace(go.Scatter(x=vect, y=elem,\
                         mode='lines', name=name,
                             opacity=alphas[count],\
                        line=dict(color=colors[count], width=widths[count])))
    count +=1
    
fig.add_trace(go.Scatter(x=vect, y=Ns,\
                         mode='lines', name='Population',
                        line=dict(color='lightblue', width=4, dash='dash')))
fig.update_layout(title='SEIRD Model',
                   xaxis_title='Time(days)',
                   yaxis_title='Cases',
                 title_x=0.5,
                  width=800,
                  height=600)

fig.show()

## Lockdown Example: New Zealand

In [128]:
new_zealand = pd.read_csv('data/master_data/new_zealand/from_wiki.csv', index_col=0)

In [129]:
def plot_cases(df):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df.index, y=np.log(df['cases'].values),\
                             mode='markers' ))

    fig.update_layout(title='Log(Infected) over time',
                       xaxis_title='Time',
                       yaxis_title='log(I)',
                     title_x=0.5,
                      width=800,
                      height=600)

    return fig

In [130]:
plot_cases(new_zealand)

In [131]:
idx_range = ('2020-03-29', '2020-04-08')

In [132]:
def get_model_r_0(df, idx):
    subset = df.loc[idx[0]:idx[1]]

    X = np.arange(1, len(subset) + 1)
    y = np.log(subset['cases'])

    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()
    return model

In [133]:
model_nz = get_model_r_0(new_zealand, idx_range)

In [134]:
R_0_end = (model_nz.params[1] + g_0)/g_0

In [135]:
R_0_start = R_0

In [136]:
L = 24
k = 0.5

In [137]:
def logistic_R_0(t):
    return (R_0_start - R_0_end)/(1 + np.exp(-k*(-t+L))) + R_0_end

In [138]:
def beta(t):
    return (logistic_R_0(t) * g_0)

In [139]:
def model_lockdown(y, t, N, b, a, f, p, g, d, mu ):
    S, E, I0, I1, I2, I3, R, D = y
    b = np.array([beta(t), beta(t), 0, 0])
    dy = [0]*8
    
    dy[0] = -np.dot(b, np.array([I0, I1, I2, I3]))*S/N
    dy[1] = np.dot(b, np.array([I0, I1, I2, I3]))*S/N - a*E
    dy[2] = a*f*E - g[0]*I0
    dy[3] = a*(1 - f)*E - g[1]*d[0]*I1 - p[0]*(1 - d[0])*I1
    dy[4] = p[0]*(1 - d[0])*I1 - g[2]*d[1]*I2 - p[1]*(1 - d[1])*I2
    dy[5] = p[1]*(1 - d[2])*I2 - g[3]*d[2]*I3 - mu*(1 - d[2])*I3
    dy[6] = np.dot(g, np.array([I0, d[0]*I1, d[1]*I2, d[2]*I3]))
    dy[7] = mu*(1 - d[2])*I3
    
    return dy

In [140]:
### Initial parameters
S, E, I0, I1, I2, I3, R, D = N-1, 1, 0, 0, 0, 0, 0, 0
y0 = S, E, I0, I1, I2, I3, R, D
max_t = 365 * 2
vect = np.linspace(0, max_t, max_t)
Ns = np.ones_like(vect) * N

ret = odeint(model_lockdown, y0, vect, args=(N, b, a, f, p, g, d, mu))

S, E, I0, I1, I2, I3, R, D = ret.T

In [141]:
def plot_sim():
    fig = go.Figure()
    labels = ['Susceptible', 'Exposed', 'Asymptomatic infections', 'mild infections', 'severe infections', \
    'critical infections', 'recovered', 'dead', 'Total infections', 'Require Hospitalization']
    colors = ['orange', 'red', 'red', 'red', 'darkred', 'darkred', 'green', 'darkred', 'red', 'darkred']
    alphas = [0.8, 0.2, 0.35, 0.7, 0.7, 0.8, 0.8, 0.8, 0.9, 1.0]
    widths = [2]* len(labels)
    widths[-1] = 4
    total_infected = I0 + I1 + I2 + I3
    require_hospitalization = I2 + I3
    count = 0
    for elem, name in zip([S, E, I0, I1, I2, I3, R, D, total_infected, require_hospitalization], labels):
        fig.add_trace(go.Scatter(x=vect, y=elem,\
                             mode='lines', name=name,
                                 opacity=alphas[count],\
                            line=dict(color=colors[count], width=widths[count])))
        count +=1

    fig.add_trace(go.Scatter(x=vect, y=Ns,\
                             mode='lines', name='Population',
                            line=dict(color='lightblue', width=4, dash='dash')))
    fig.update_layout(title='SEIRD Model',
                       xaxis_title='Time(days)',
                       yaxis_title='Cases',
                     title_x=0.5,
                      width=800,
                      height=600)

    fig.show()

In [142]:
plot_sim()

### Widespread Early Detection: South Korea

In [143]:
south_korea = pd.read_csv('data/master_data/south_korea/from_wiki.csv', index_col=0)

In [144]:
plot_cases(south_korea)

In [145]:
idx = ('2020-03-01', '2020-03-16')
model_sk = get_model_r_0(south_korea, idx)

In [146]:
R_0_end = b_0/(b_0 - model_sk.params[1] )

In [147]:
def gamma(t):
    return b_0/logistic_R_0(t)

In [148]:
def model_early_detection(y, t, N, b, a, f, p, g, d, mu ):
    S, E, I0, I1, I2, I3, R, D = y
    g = np.array([gamma(t), gamma(t), g_2, g_3])
    dy = [0]*8
    
    dy[0] = -np.dot(b, np.array([I0, I1, I2, I3]))*S/N
    dy[1] = np.dot(b, np.array([I0, I1, I2, I3]))*S/N - a*E
    dy[2] = a*f*E - g[0]*I0
    dy[3] = a*(1 - f)*E - g[1]*d[0]*I1 - p[0]*(1 - d[0])*I1
    dy[4] = p[0]*(1 - d[0])*I1 - g[2]*d[1]*I2 - p[1]*(1 - d[1])*I2
    dy[5] = p[1]*(1 - d[2])*I2 - g[3]*d[2]*I3 - mu*(1 - d[2])*I3
    dy[6] = np.dot(g, np.array([I0, d[0]*I1, d[1]*I2, d[2]*I3]))
    dy[7] = mu*(1 - d[2])*I3
    
    return dy

In [149]:
### Initial parameters
S, E, I0, I1, I2, I3, R, D = N-1, 1, 0, 0, 0, 0, 0, 0
y0 = S, E, I0, I1, I2, I3, R, D
max_t = 365 * 3
vect = np.linspace(0, max_t, max_t)
Ns = np.ones_like(vect) * N

ret = odeint(model_early_detection, y0, vect, args=(N, b, a, f, p, g, d, mu))

S, E, I0, I1, I2, I3, R, D = ret.T

In [150]:
plot_sim()